In [39]:
# Librerias

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from datetime import datetime, timedelta, time
import json

In [49]:
def graph(json_file_in, json_file_out, initial_hour):
    with open(json_file_out, 'r') as file:
        data = json.load(file)

    grids_df = pd.DataFrame(data['grids']) if 'grids' in data else pd.DataFrame()

    demands_df = pd.DataFrame(data['demands']) if 'demands' in data else pd.DataFrame()

    tssas_df = pd.DataFrame(data['tssas']) if 'tssas' in data else pd.DataFrame()

    besss_df = pd.DataFrame(data['besss']) if 'besss' in data else pd.DataFrame()

    local_sources_df = pd.DataFrame(data['local_sources']) if 'local_sources' in data else pd.DataFrame()

    electric_cars_df = pd.DataFrame(data['electric_cars']) if 'electric_cars' in data else pd.DataFrame()

    cesas_df = pd.DataFrame(data['cesas']) if 'cesas' in data else pd.DataFrame()

    grids_tariff_df = pd.DataFrame(data['grids']) if 'grids' in data else pd.DataFrame()

    with open(json_file_in, 'r') as file:
        data = json.load(file)
    
    grids_tariff_IN_df = pd.DataFrame(data['grids']) if 'grids' in data else pd.DataFrame()

    cesas_IN_df = pd.DataFrame(data['cesas']) if 'cesas' in data else pd.DataFrame()

    tssas_IN_df = pd.DataFrame(data['tssas']) if 'tssas' in data else pd.DataFrame()

    # Tarifa

    initial_time = datetime.strptime(initial_hour, "%H:%M")
    num_values = len(grids_tariff_IN_df['energy_buy_price_sched'][0])
    
 
    hours = [initial_time + timedelta(hours=i) for i in range(num_values)]
    energy_tariffs = grids_tariff_IN_df['energy_buy_price_sched'][0]
    power_factors = grids_tariff_IN_df['power_factor_sched'][0]


    hours_str = []
    day_counter = 1
    for i in range(num_values):
        if i % 24 == 0 and i != 0:
            day_counter += 1
        hours_str.append(f'Día {day_counter} {hours[i].strftime("%H:%M")}')

    fig = go.Figure()

    segment_start = 0
    current_color = 'blue'

    for i in range(1, num_values):
        new_color = 'red' if power_factors[i] == 1 else 'blue'
        
        if new_color != current_color:
            fig.add_trace(go.Scatter(
                x=hours_str[segment_start:i+1],
                y=energy_tariffs[segment_start:i+1],
                mode='lines',
                line=dict(color=current_color),
                name='Energy Tariffs' if segment_start == 0 else '',
                showlegend=segment_start == 0
            ))
            segment_start = i
            current_color = new_color

    fig.add_trace(go.Scatter(
        x=hours_str[segment_start:num_values],
        y=energy_tariffs[segment_start:num_values],
        mode='lines',
        line=dict(color=current_color),
        name='Energy Tariffs' if segment_start == 0 else '',
        showlegend=segment_start == 0
    ))

    fig.update_layout(
        title='Precio energía',
        xaxis_title='Tiempo',
        yaxis_title='CLP/kWh',
        legend_title='Leyenda',
        xaxis=dict(tickangle=90),
        template='plotly_white'
    )
    fig.show()

    # Baterías

    num_values = len(besss_df['efin_values'][0])
    hours = [initial_time + timedelta(hours=i) for i in range(num_values)]

    efin_values = besss_df['efin_values'][0]
    inflow_values = besss_df['flow_in_values'][0]
    outflow_values = [-value for value in besss_df['flow_out_values'][0]] 

    # Convertir horas a string con etiqueta de días
    hours_str = []
    day_counter = 1
    for i in range(num_values):
        if i % 24 == 0 and i != 0:
            day_counter += 1
        hours_str.append(f'Día {day_counter} {hours[i].strftime("%H:%M")}')

    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=hours_str,
        y=efin_values,
        mode='lines',
        name='Efin Values'
    ))

    fig.add_trace(go.Scatter(
        x=hours_str,
        y=inflow_values,
        mode='lines',
        name='Inflow Values'
    ))

    fig.add_trace(go.Scatter(
        x=hours_str,
        y=outflow_values,
        mode='lines',
        name='Outflow Values'
    ))

    fig.update_layout(
        title='BESSS',
        xaxis_title='Tiempo',
        yaxis_title='kW',
        legend_title='Leyenda',
        xaxis=dict(tickangle=90),
        template='plotly_white'
    )

    fig.show()

    # Consumos - inyecciones

    num_values = len(demands_df['load_values'][0])
    hours = [initial_time + timedelta(hours=i) for i in range(num_values)]

    hours_str = []
    day_counter = 1
    for i in range(num_values):
        if i % 24 == 0 and i != 0:
            day_counter += 1
        hours_str.append(f'Día {day_counter} {hours[i].strftime("%H:%M")}')

    load_values = demands_df['load_values'][0]

    power_values = {}
    for index, row in tssas_IN_df.iterrows():
        tssa_name = row['name']
        power_values[tssa_name] = row['load']

    for index, row in cesas_IN_df.iterrows():
        cesas_name = row['name']
        power_values[cesas_name] = row['load']

    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=hours_str,
        y=load_values,
        mode='lines',
        name='Consumo: d1'
    ))

    total_values = [0] * num_values

    total_injection_values = [0] * num_values
    for index, row in tssas_df.iterrows():
        tssa_name = row['name']
        tssa_values = [value * power_values[tssa_name] for value in row['onoff_values']]
        fig.add_trace(go.Scatter(
            x=hours_str,
            y=tssa_values,
            mode='lines',
            name=f'Consumo: {tssa_name}'
        ))

        total_values = [total + tssa for total, tssa in zip(total_values, tssa_values)]


    for index, row in electric_cars_df.iterrows():
        car_name = row['name']
        flow_in = row['battery_flow_in_values']
        fig.add_trace(go.Scatter(
            x=hours_str,
            y=flow_in,
            mode='lines',
            name=f'Consumo: {car_name}'
        ))

        total_values = [total + ves for total, ves in zip(total_values, flow_in)]

    for index, row in cesas_df.iterrows():
        cesas_name = row['name']
        flow_cesas = row['onoff_values']
        flow_cesas = pd.Series(flow_cesas) * 2
        fig.add_trace(go.Scatter(
            x=hours_str,
            y=flow_cesas,
            mode='lines',
            name=f'Consumo: {cesas_name}'
        ))

        total_values = [total + ves for total, ves in zip(total_values, flow_cesas)]

    total_values = [total + load for total, load in zip(total_values, load_values)]

    fig.add_trace(go.Scatter(
        x=hours_str,
        y=total_values,
        mode='lines',
        name='Consumo total'
    ))
    for index, row in local_sources_df.iterrows():
        injection_values = row['generation_values']
        fig.add_trace(go.Scatter(
            x=hours_str,
            y=injection_values,
            mode='lines',
            name=f'Inyección: {row["name"]}'
        ))
        total_injection_values = [total + injection for total, injection in zip(total_injection_values, injection_values)]

    for index, row in besss_df.iterrows():
        outflow_values = row['flow_out_values']
        fig.add_trace(go.Scatter(
            x=hours_str,
            y=outflow_values,
            mode='lines',
            name=f'Inyección: {row["name"]}'
        ))

        total_injection_values = [total + outflow for total, outflow in zip(total_injection_values, outflow_values)]

    for index, row in grids_tariff_df.iterrows():
        injection_values = row['withdrawn_values']
        fig.add_trace(go.Scatter(
            x=hours_str,
            y=injection_values,
            mode='lines',
            name=f'Inyección: {row["name"]}'
        ))

        total_injection_values = [total + injection for total, injection in zip(total_injection_values, injection_values)]
    fig.add_trace(go.Scatter(
        x=hours_str,
        y=total_injection_values,
        mode='lines',
        name='Inyección total'
    ))
    fig.update_layout(
        title='Resultados',
        xaxis_title='Tiempo',
        yaxis_title='kW',
        legend_title='Leyenda',
        xaxis_rangeslider_visible=True,
        xaxis=dict(tickangle=90),
        template='plotly_white'
    )
    fig.show()


In [51]:
json_file_in = 'Input/fullweek.json'   # json de entrada a gilda_opts
json_file_out = 'Input/fullweek-output.json'   # json con la solución del optimizador
initial_hour = "01:00"                         # Hora inicial de la ventana de tiempo
graph(json_file_in,json_file_out,initial_hour)